# Заголовок и метаданные
"""
# Case Study по модулю SQL
# Имя Фамилия: Сафарова Гулрухсор
# Дата выполнения: 23 марта 2025
"""

In [22]:
import psycopg2
import pandas as pd

# Создание подключения к базе данных PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",   
    user="Gulrukhsor",         
    password="123", 
    host="localhost",         
    port="5432"               
)

# Функция для выполнения SQL-запросов
def run_query(query):
    with conn.cursor() as cursor:
        cursor.execute(query)
        conn.commit()

OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "Gulrukhsor"


# Блок 1. Создание схемы и таблиц
# В базе данных создайте новую схему adv_works. На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.

In [23]:
# Создание схемы adv_works
create_schema_query = """
CREATE SCHEMA IF NOT EXISTS adv_works;
"""
run_query(create_schema_query)

# Создание таблиц в схеме adv_works
create_tables_queries = [
    """
    CREATE TABLE IF NOT EXISTS adv_works.products (
        product_id SERIAL PRIMARY KEY,
        product_name VARCHAR(255),
        category_id INT,
        price DECIMAL(10, 2)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS adv_works.product_categories (
        category_id SERIAL PRIMARY KEY,
        category_name VARCHAR(255)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS adv_works.customers (
        customer_id SERIAL PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(255) UNIQUE,
        occupation VARCHAR(100),
        yearly_income DECIMAL(10, 2),
        marital_status VARCHAR(50),
        has_children BOOLEAN
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS adv_works.orders (
        order_id SERIAL PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount DECIMAL(10, 2),
        FOREIGN KEY (customer_id) REFERENCES adv_works.customers(customer_id)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS adv_works.order_details (
        order_detail_id SERIAL PRIMARY KEY,
        order_id INT,
        product_id INT,
        quantity INT,
        total_price DECIMAL(10, 2),
        FOREIGN KEY (order_id) REFERENCES adv_works.orders(order_id),
        FOREIGN KEY (product_id) REFERENCES adv_works.products(product_id)
    );
    """
]

# Выполнение запросов для создания таблиц
for query in create_tables_queries:
    run_query(query)


NameError: name 'conn' is not defined

# Блок 2. Аналитические задачи

# Секция 1. Анализ клиентов
# Сегментация по доходу: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.
# Семейный профиль: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.
# Высокодоходные клиенты: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле SalesAmount). Итоговая таблица должна содержать следующие поля: customer_key, customer_name, total_purchase.
# Влияние семейного положения: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (MaritalStatus) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: year, marital_status, avg_sales_amount.

In [24]:
# Сегментация по доходу
SELECT occupation, 
       COUNT(customer_id) AS number_of_customers,
       AVG(yearly_income) AS avg_income
FROM adv_works.customers
GROUP BY occupation;


SyntaxError: invalid syntax (3746502569.py, line 2)

In [25]:
# Семейный профиль
SELECT has_children, 
       COUNT(customer_id) * 100.0 / (SELECT COUNT(*) FROM adv_works.customers) AS pct_of_customer_base
FROM adv_works.customers
GROUP BY has_children;

SyntaxError: invalid syntax (2115600594.py, line 2)

In [26]:
# Высокодоходные клиенты
SELECT c.customer_id, 
       CONCAT(c.first_name, ' ', c.last_name) AS customer_name, 
       SUM(od.total_price) AS total_purchase
FROM adv_works.customers c
JOIN adv_works.orders o ON c.customer_id = o.customer_id
JOIN adv_works.order_details od ON o.order_id = od.order_id
GROUP BY c.customer_id
ORDER BY total_purchase DESC
LIMIT 10;

SyntaxError: invalid syntax (468658964.py, line 2)

In [27]:
# Влияние семейного положения
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       c.marital_status, 
       AVG(od.total_price) AS avg_sales_amount
FROM adv_works.orders o
JOIN adv_works.customers c ON o.customer_id = c.customer_id
JOIN adv_works.order_details od ON o.order_id = od.order_id
GROUP BY year, c.marital_status;

SyntaxError: invalid syntax (2666216000.py, line 2)

# Секция 2. Анализ продаж
# Ежемесячные продажи: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, sales_count (количество продаж), sales_amount.
# Продажи по регионам: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: region, sales_count, sales_amount.

In [28]:
# Ежемесячные продажи
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       EXTRACT(MONTH FROM o.order_date) AS monthkey,
       TO_CHAR(o.order_date, 'Month') AS month_name,
       COUNT(o.order_id) AS sales_count,
       SUM(od.total_price) AS sales_amount
FROM adv_works.orders o
JOIN adv_works.order_details od ON o.order_id = od.order_id
WHERE EXTRACT(YEAR FROM o.order_date) IN (2003, 2004)
GROUP BY year, monthkey
ORDER BY year, monthkey;

SyntaxError: invalid syntax (231156942.py, line 2)

In [29]:
# Продажи по регионам
SELECT r.region_name, 
       COUNT(o.order_id) AS sales_count, 
       SUM(od.total_price) AS sales_amount
FROM adv_works.orders o
JOIN adv_works.order_details od ON o.order_id = od.order_id
JOIN adv_works.regions r ON o.region_id = r.region_id
GROUP BY r.region_name;

SyntaxError: invalid syntax (533901263.py, line 2)

# Секция 3. Анализ продуктов
# Доля продаж: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: year, product_key, product_category_key, english_product_category_name, sales_amount, pct_of_total_sales.
# Самые продаваемые продукты: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: product_key, product_name, english_product_category_name, sales_amount
# Маржа от продаж: Посчитайте разницу между суммой продаж (SalesAmount) за минусом себестоимости (TotalProductCost), налогов (*TaxAmt) и расходов на доставку (Freight) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, product_key, product_name, sales_amount, total_product_cost, tax_amt, freight, margin, margin_pct (маржа как процент от суммы продаж).

In [30]:
# Доля продаж каждой категории продуктов
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       p.product_id AS product_key,
       p.category_id AS product_category_key,
       c.category_name AS english_product_category_name,
       SUM(od.total_price) AS sales_amount,
       (SUM(od.total_price) * 100.0 / 
        (SELECT SUM(od2.total_price) FROM adv_works.order_details od2)) AS pct_of_total_sales
FROM adv_works.products p
JOIN adv_works.order_details od ON p.product_id = od.product_id
JOIN adv_works.product_categories c ON p.category_id = c.category_id
JOIN adv_works.orders o ON od.order_id = o.order_id
GROUP BY year, product_key, product_category_key, english_product_category_name
ORDER BY sales_amount DESC;

SyntaxError: invalid syntax (1716410731.py, line 2)

In [31]:
# Самые продаваемые продукты
SELECT p.product_id AS product_key, 
       p.product_name, 
       c.category_name AS english_product_category_name, 
       SUM(od.total_price) AS sales_amount
FROM adv_works.products p
JOIN adv_works.order_details od ON p.product_id = od.product_id
JOIN adv_works.product_categories c ON p.category_id = c.category_id
JOIN adv_works.orders o ON od.order_id = o.order_id
GROUP BY product_key, p.product_name, c.category_name
ORDER BY sales_amount DESC
LIMIT 5;

SyntaxError: invalid syntax (1757015013.py, line 2)

In [32]:
# Маржа от продаж по продуктам
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       EXTRACT(MONTH FROM o.order_date) AS monthkey,
       TO_CHAR(o.order_date, 'Month') AS month_name,
       p.product_id AS product_key,
       p.product_name,
       SUM(od.total_price) AS sales_amount,
       SUM(p.total_product_cost) AS total_product_cost,
       SUM(o.tax_amt) AS tax_amt,
       SUM(o.freight) AS freight,
       (SUM(od.total_price) - SUM(p.total_product_cost) - SUM(o.tax_amt) - SUM(o.freight)) AS margin,
       ((SUM(od.total_price) - SUM(p.total_product_cost) - SUM(o.tax_amt) - SUM(o.freight)) * 100.0 / SUM(od.total_price)) AS margin_pct
FROM adv_works.products p
JOIN adv_works.order_details od ON p.product_id = od.product_id
JOIN adv_works.orders o ON od.order_id = o.order_id
GROUP BY year, monthkey, month_name, product_key, p.product_name
ORDER BY year, monthkey, sales_amount DESC;

SyntaxError: invalid syntax (4116512290.py, line 2)

# Секция 4. Анализ трендов
# Квартальный рост: Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: year, quarter_id, product_category_key, english_product_category_name, quarter_sales_amount, quarter_over_quarter_growth_pct.
# Сравнение будних и выходных (суббота, воскресенье) дней: Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: year, day_name, is_weekend (где 1 означает выходной а 0 будний день), sales_amount.

In [33]:
# Квартальный рост
WITH top_categories AS (
    SELECT p.category_id AS product_category_key,
           c.category_name AS english_product_category_name,
           SUM(od.total_price) AS total_sales
    FROM adv_works.products p
    JOIN adv_works.order_details od ON p.product_id = od.product_id
    JOIN adv_works.product_categories c ON p.category_id = c.category_id
    GROUP BY p.category_id, c.category_name
    ORDER BY total_sales DESC
    LIMIT 2
)
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       EXTRACT(QUARTER FROM o.order_date) AS quarter_id,
       p.category_id AS product_category_key,
       c.category_name AS english_product_category_name,
       SUM(od.total_price) AS quarter_sales_amount,
       ROUND(
           (SUM(od.total_price) - LAG(SUM(od.total_price)) OVER (PARTITION BY p.category_id ORDER BY EXTRACT(YEAR FROM o.order_date), EXTRACT(QUARTER FROM o.order_date))) 
           * 100.0 / 
           LAG(SUM(od.total_price)) OVER (PARTITION BY p.category_id ORDER BY EXTRACT(YEAR FROM o.order_date), EXTRACT(QUARTER FROM o.order_date)),
           2
       ) AS quarter_over_quarter_growth_pct
FROM adv_works.products p
JOIN adv_works.order_details od ON p.product_id = od.product_id
JOIN adv_works.orders o ON od.order_id = o.order_id
JOIN top_categories c ON p.category_id = c.product_category_key
GROUP BY year, quarter_id, product_category_key, english_product_category_name
ORDER BY year, quarter_id, quarter_sales_amount DESC;

SyntaxError: invalid syntax (2352194348.py, line 2)

In [34]:
# Сравнение будних и выходных дней
SELECT EXTRACT(YEAR FROM o.order_date) AS year,
       TO_CHAR(o.order_date, 'Day') AS day_name,
       CASE 
           WHEN TO_CHAR(o.order_date, 'Day') IN ('Saturday', 'Sunday') THEN 1
           ELSE 0
       END AS is_weekend,
       SUM(od.total_price) AS sales_amount
FROM adv_works.orders o
JOIN adv_works.order_details od ON o.order_id = od.order_id
GROUP BY year, day_name, is_weekend
ORDER BY year, is_weekend DESC, sales_amount DESC;

SyntaxError: invalid syntax (3742317905.py, line 2)